In [2]:
import importlib.util
import subprocess

# Lista de paquetes que queremos instalar si no están presentes
packages = [
    "numpy",
    "pandas",
    "skforecast"
]

# Función para verificar e instalar paquetes
def install_if_missing(package):
    if importlib.util.find_spec(package) is None:
        print(f"Instalando {package}...")
        subprocess.check_call(["pip", "install", package])
    else:
        print(f"{package} ya está instalado.")

# Iterar sobre la lista de paquetes
for package in packages:
    install_if_missing(package)
    # Importamos las librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# Importamos librerías para visualización
from seaborn import heatmap
from matplotlib import pyplot as plt

# Análisis y estadísticas

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from scipy import stats
from scipy.stats import boxcox

# Machine Learning y preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Forecasting con Skforecast
import skforecast
from skforecast.datasets import fetch_dataset
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import TimeSeriesFold, grid_search_forecaster, backtesting_forecaster
from skforecast.preprocessing import RollingFeatures
from skforecast.utils import save_forecaster, load_forecaster

# SHAP para interpretabilidad de modelos
import shap
import joblib


numpy ya está instalado.
pandas ya está instalado.
skforecast ya está instalado.


Tras Varias pruebas, por error, captura de tendencia, y coste computacional vamos a utilizar el modelo ridge multipaso directo para la prediccion de la net generation por unidad operativa

si se quisiera calcular el resto de variables mejor SARIMAX


In [3]:
df=pd.read_excel('/content/monthly_data_per_unit.xlsx',index_col=0)
df.index = pd.to_datetime(df.index).to_period('M')
df.head(),df.index

(         Net_Summer_Capacity_MW  Net_Generation_MWh  \
 Date                                                  
 1994-12                0.909615          555.018349   
 1995-01                0.909615          581.119266   
 1995-02                0.909615          475.761468   
 1995-03                0.909615          475.963303   
 1995-04                0.909615          452.486239   
 
          Share_of_Electricity_Percent  Capacity_Factor_Percent  
 Date                                                            
 1994-12                      0.205505                 0.752294  
 1995-01                      0.207339                 0.788073  
 1995-02                      0.188991                 0.713761  
 1995-03                      0.182569                 0.644954  
 1995-04                      0.185321                 0.634862  ,
 PeriodIndex(['1994-12', '1995-01', '1995-02', '1995-03', '1995-04', '1995-05',
              '1995-06', '1995-07', '1995-08', '1995-09',
     

In [22]:
def escalar_xy(data, variables_exogoenas):
    """
    Escala la(s) variable(s) exogenas y extrae la variable dependiente del conjunto de datos proporcionado.

    Parámetros:
    - data (pd.DataFrame): DataFrame con los datos originales, debe contener las columnas necesarias.
    - variables (list): Lista de nombres de las variables exógenas a escalar.

    Retorna:
    - X_escalado (pd.DataFrame): DataFrame con las variables exógenas escaladas.
    - Y_data (pd.Series): Serie con la variable dependiente "Net_Generation_MWh".
    - scalers (dict): Diccionario con los `StandardScaler()` usados para cada variable.
    """
    scalers = {}  # Diccionario para almacenar los scalers

    # Extraer y escalar cada variable exógena individualmente
    X_escalado = pd.DataFrame(index=data.index)

    for col in variables_exogoenas:
        scaler = StandardScaler()
        X_escalado[col] = scaler.fit_transform(data[[col]])  # Escalar la columna
        scalers[col] = scaler  # Guardar el scaler usado



    # Seleccionar la variable dependiente
    Y_data = data["Net_Generation_MWh"]
    Y_data.index = X_escalado.index  # Asignar el índice de X_escalado

    return X_escalado, Y_data, scalers


def entrenar_modelos_sarimax(X_escalado, variables, sarima_order=(2,1,2), seasonal_order=(1,1,1,12)):
    """
    Entrena modelos SARIMAX para cada variable exógena seleccionada.

    Parámetros:
    - X_escalado (pd.DataFrame): DataFrame con las variables exógenas escaladas.
    - variables (list): Lista de nombres de las variables a modelar.
    - sarima_order (tuple): Parámetros (p, d, q) del modelo SARIMAX.
    - seasonal_order (tuple): Parámetros estacionales (P, D, Q, S) del modelo SARIMAX.

    Retorna:
    - modelos_sarimax (dict): Diccionario con los modelos entrenados para cada variable.
    """
    modelos_sarimax = {}

    # Iterar sobre cada variable exógena seleccionada y entrenar un SARIMAX distinto
    for var in variables:
        print(f"📊 Entrenando modelo SARIMAX para {var}...")

        # Definir modelo SARIMAX para la variable actual
        model = SARIMAX(
            X_escalado[var],  # Modelamos la variable específica
            order=sarima_order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit()

        # Guardar el modelo en el diccionario
        modelos_sarimax[var] = model

    return modelos_sarimax


def entrenar_modelo_ridge(Y_data, años, lags):
    """
    Entrena un modelo Ridge con Skforecast para una serie temporal dada.

    Parámetros:
    - Y_data (pd.Series): Serie temporal que se usará como variable dependiente.
    - años (int): Cantidad de años para la predicción.
    - lags (list o array): Lags a utilizar en el modelo.

    Retorna:
    - forecaster_ridge (ForecasterDirect): Modelo entrenado listo para predecir.
    """

    Y_data.index = Y_data.index.to_timestamp()
    Y_data = Y_data.asfreq('MS')





    # Verificar que el índice es correcto ahora
    print("\n🔍 Índice de Y_data después de la conversión:")
    print(Y_data.index)
    print(f"Frecuencia del índice después de la conversión: {Y_data.index.freq}")

    # Definir y entrenar el modelo Ridge con Skforecast
    forecaster_ridge = ForecasterDirect(
        regressor=Ridge(),
        lags=lags,
        steps=12 * años
    )

    # Entrenar el modelo
    forecaster_ridge.fit(Y_data)

    return forecaster_ridge





In [ ]:
#rm SARIMAX_Net_Summer_Capacity_MW.joblib SARIMAX_Capacity_Factor_Percent.joblib modelo_ridge_NetGeneration.joblib scalers.pkl X_escalado.xlsx Y_escalado.xlsx


In [ ]:
#Entrenar y guardar modelos SARIMAX con Skforecast
modelos_sarimax = entrenar_modelos_sarimax(X_escalado, ['Net_Summer_Capacity_MW', 'Capacity_Factor_Percent'],
                                           sarima_order=(2,1,2), seasonal_order=(1,1,1,12))

SARIMAX_Net_Summer_Capacity_MW = modelos_sarimax['Net_Summer_Capacity_MW']
SARIMAX_Capacity_Factor_Percent = modelos_sarimax['Capacity_Factor_Percent']

In [27]:
#Escalar los datos
X_escalado, Y_escalado, scalers = escalar_xy(df, ['Net_Summer_Capacity_MW', 'Capacity_Factor_Percent'])
X_escalado.head(), Y_escalado.head(), scalers
print(Y_escalado,Y_escalado.index,type(Y_escalado))

#Guardar los scalers con joblib
joblib.dump(scalers, "scalers.pkl")
print("Scalers guardados en scalers.pkl")


#Entrenar y guardar modelo Ridge con Skforecast
años = 15
lags = np.arange(1, 35)
modelo_ridge_NetGeneration = entrenar_modelo_ridge(Y_escalado, años, lags)

# Guardar con save_forecaster()
joblib.dump(modelo_ridge_NetGeneration, "modelo_ridge_NetGeneration.pkl")
print("Modelo Ridge guardado en modelo_ridge_NetGeneration.joblib")
save_forecaster(SARIMAX_Net_Summer_Capacity_MW, "SARIMAX_Net_Summer_Capacity_MW.joblib")
save_forecaster(SARIMAX_Capacity_Factor_Percent, "SARIMAX_Capacity_Factor_Percent.joblib")
save_forecaster(modelo_ridge_NetGeneration, "modelo_ridge_NetGeneration.joblib")

Date
1994-12    555.018349
1995-01    581.119266
1995-02    475.761468
1995-03    475.963303
1995-04    452.486239
              ...    
2024-07    743.457447
2024-08    742.127660
2024-09    666.595745
2024-10    621.670213
2024-11    658.553191
Freq: M, Name: Net_Generation_MWh, Length: 360, dtype: float64 PeriodIndex(['1994-12', '1995-01', '1995-02', '1995-03', '1995-04', '1995-05',
             '1995-06', '1995-07', '1995-08', '1995-09',
             ...
             '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07',
             '2024-08', '2024-09', '2024-10', '2024-11'],
            dtype='period[M]', name='Date', length=360) <class 'pandas.core.series.Series'>
Scalers guardados en scalers.pkl

🔍 Índice de Y_data después de la conversión:
DatetimeIndex(['1994-12-01', '1995-01-01', '1995-02-01', '1995-03-01',
               '1995-04-01', '1995-05-01', '1995-06-01', '1995-07-01',
               '1995-08-01', '1995-09-01',
               ...
               '2024-02-0

In [24]:
from skforecast.direct import ForecasterDirect
from sklearn.linear_model import Ridge



X_escalado, Y_escalado, scalers = escalar_xy(df, ['Net_Summer_Capacity_MW', 'Capacity_Factor_Percent'])
X_escalado.head(), Y_escalado.head(), scalers

modelo_ridge_NetGeneration = entrenar_modelo_ridge(Y_escalado, años=15, lags=np.arange(1, 35))
prediction = modelo_ridge_NetGeneration.predict(steps=120)
prediction, modelo_ridge_NetGeneration


🔍 Índice de Y_data después de la conversión:
DatetimeIndex(['1994-12-01', '1995-01-01', '1995-02-01', '1995-03-01',
               '1995-04-01', '1995-05-01', '1995-06-01', '1995-07-01',
               '1995-08-01', '1995-09-01',
               ...
               '2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01',
               '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01',
               '2024-10-01', '2024-11-01'],
              dtype='datetime64[ns]', name='Date', length=360, freq='MS')
Frecuencia del índice después de la conversión: <MonthBegin>


(2024-12-01    721.222239
 2025-01-01    735.295695
 2025-02-01    668.332680
 2025-03-01    659.026093
 2025-04-01    591.621098
                  ...    
 2034-07-01    741.851479
 2034-08-01    723.441195
 2034-09-01    663.604319
 2034-10-01    628.529392
 2034-11-01    671.840880
 Freq: MS, Name: pred, Length: 120, dtype: float64,
 ForecasterDirect 
 Regressor: Ridge 
 Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
  25 26 27 28 29 30 31 32 33 34] 
 Window features: None 
 Window size: 34 
 Maximum steps to predict: 180 
 Exogenous included: False 
 Exogenous names: None 
 Transformer for y: None 
 Transformer for exog: None 
 Weight function included: False 
 Differentiation order: None 
 Training range: [Timestamp('1994-12-01 00:00:00'), Timestamp('2024-11-01 00:00:00')] 
 Training index type: DatetimeIndex 
 Training index frequency: MS 
 Regressor parameters: 
     {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None,
     'pos

In [82]:
#Guardamos X e Y
X_escalado.to_excel('X_escalado.xlsx')
Y_escalado.to_excel('Y_escalado.xlsx')

In [83]:
# Descargar cada archivo individualmente
from google.colab import files
files.download("SARIMAX_Net_Summer_Capacity_MW.joblib")
files.download("SARIMAX_Capacity_Factor_Percent.joblib")
files.download("modelo_ridge_NetGeneration.joblib")
files.download("scalers.pkl")
files.download("X_escalado.xlsx")
files.download("Y_escalado.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
import pickle

# Guardar el modelo completo
with open("modelo_ridge_pickle.pkl", "wb") as f:
    pickle.dump(modelo_ridge_NetGeneration, f)
print("\n✅ Modelo guardado con pickle.")



✅ Modelo guardado con pickle.
